# Notebook 2: Application d'analyse immobilière
---


**Projet développé par :**
- Ashley OHNONA
- Harisoa RANDRIANASOLO
- Fairouz YOUDARENE
- Jennifer ZAHORA

**Date :** 3/11/2025

---

## Présentation du projet
### Persona ciblé

#### Profil
**Nom**: Marie
**Age**: 40 ans
**Occupation**: cadre supérieur à Paris

**Budget**: 400-600k€

**Objectif**: Acheter son premier bien locatif pour générer des revenus passifs


#### Besoins spécifiques
1. Avoir un aperçu clair du marché (prix au m2,tendances) (dashboard)
2. Identifier les villes avec le meilleur rendement locatif dans IDF
3. Comprendre quels types de biens sont les plus rentables (par département, par commune)
4. Comparer le potentiel de location simple vs coloc
5. Évaluer la liquidité du marché
6. Analyser des accessibilité des transports en commun (par département, par commune)
7. Vérifier la sécurité et l’attractivité du quartier
8. Trouver des biens accessibles avec son budget (par département, par commune)
9. Obtenir un classement des Top 10 opportunités actuelles (un tableau ou score automatique). 



---

## Objectifs de l'application
Identifier des territoires prometteurs pour un premier investissement locatif

---

## Sources de données
- DVF2024 : (mettre le lien pour téléchargemet direct)
- Transport : [emplacement-des-gares-idf.csv](https://data.iledefrance-mobilites.fr/api/explore/v2.1/catalog/datasets/emplacement-des-gares-idf/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B)
- Données Loyers : (mettre le lien pour téléchargemet direct)
---

## Import des bibliothèques

In [1]:
import pandas as pd
import numpy as np
from ipywidgets import interact, Dropdown, IntSlider
from ipywidgets import interact, IntSlider
import plotly.express as px


---

### Chargement des données

In [2]:
# --- Chargement des données nettoyées (préparées dans le Notebook 1) ---
df = pd.read_csv("dvf_idf_clean.csv")



### Widget 1 :  Avoir un aperçu clair du marché (prix au m2,tendances) (dashboard)
- Widget  à part par ce que elle est faite sur streamlit

### Widget 2 : Identifier les villes avec le meilleur rendement locatif dans IDF
<!-- COMPLÉTEZ ICI: Décrivez les paramètres -->
<!-- Exemple: Prix d'achat, loyer mensuel estimé, charges, etc. -->


In [ ]:
# CODEZ ICI: Créer les inputs interactifs

# CODEZ ICI: Fonction de calcul du ROI, rendement brut, rendement net

# CODEZ ICI: Affichage des résultats avec visualisation

### Widget 3 : Comprendre quels types de biens sont les plus rentables (par département, par commune)

<!-- COMPLÉTEZ ICI: Décrivez les fonctionnalités de la carte -->
<!-- Exemple: -->
<!-- - Carte choroplèthe des prix moyens -->
<!-- - Filtres par type de bien -->
<!-- - Pop-up avec détails au clic -->
<!-- - Layers pour différentes métriques -->

In [ ]:
# CODEZ ICI: Créer la carte interactive avec Folium ou Plotly

# CODEZ ICI: Ajouter les paramètres de filtrage

# CODEZ ICI: Afficher la carte



### Widget 4 : Comparer le potentiel de location simple vs coloc

<!-- COMPLÉTEZ ICI: Décrivez tous les paramètres du système de recommandation -->
<!-- Exemple: -->
<!-- - Budget min/max -->
<!-- - Type(s) de bien souhaité(s) -->
<!-- - Rendement minimum attendu -->
<!-- - Région(s) préférée(s) -->
<!-- - Niveau de risque acceptable -->
<!-- - etc. -->



### Widget 5 : Évaluer la liquidité du marché — Aperçu global du marché immobilier IDF

Ce  widget fournit une vision d’ensemble du marché immobilier francilien à travers plusieurs indicateurs clés : **prix moyen et médian au m²**, **volume total de transactions**, et **surface moyenne des biens vendus**. La médiane \((P_{50})\) limite l’effet des valeurs extrêmes, tandis que la moyenne  
\[
\text{moyenne} = \frac{1}{n}\sum x_i
\]
décrit le niveau général des prix.

Afin d’évaluer la dynamique du marché, nous construisons pour chaque commune un **indice de liquidité** combinant :  
1) le **volume annuel de ventes**,  
2) la **continuité des ventes** sur l’année (nombre de mois avec au moins une transaction),  
3) la **stabilité mensuelle** mesurée via le **coefficient de variation (CV)**  
\[
CV = \frac{\sigma}{\mu}
\]

où  
- \( \sigma \) = écart-type mensuel  
- \( \mu \) = moyenne mensuelle  

L’**indice final de liquidité** est obtenu par combinaison pondérée :  
\[
\text{score} = 0.6\,(\text{volume}) + 0.2\,(\text{mois actifs}) + 0.2\,(\text{stabilité})
\]

Un score élevé traduit un marché **actif et régulier**, donc facilité pour la revente.  

​



In [3]:
# Sécuriser la date
df["date_mutation"] = pd.to_datetime(df["date_mutation"], errors="coerce")
df = df.dropna(subset=["date_mutation"])

# Colonnes minimales utiles
needed = ["date_mutation", "code_commune", "nom_commune", "latitude", "longitude"]
assert all(c in df.columns for c in needed), "Colonnes manquantes pour l'indice."

df["mois"] = df["date_mutation"].dt.to_period("M").astype(str)

# Ventes par commune × mois
mth = (df.groupby(["code_commune","nom_commune","mois"], as_index=False)
               .size()
               .rename(columns={"size":"nb_ventes_mois"}))

# On veut 12 mois même si 0 vente 
all_months = pd.period_range("2024-01", "2024-12", freq="M").astype(str)
pivot = (mth.pivot_table(index=["code_commune","nom_commune"],
                         columns="mois", values="nb_ventes_mois", fill_value=0)
            .reindex(columns=all_months, fill_value=0))

pivot.head()

feat = pivot.copy()
feat["nb_ventes_2024"] = feat.sum(axis=1)
feat["nb_mois_actifs"] = (feat[all_months] > 0).sum(axis=1)

# CV mensuel (std/mean) — si mean=0 → CV=0 pour éviter NaN
mean = feat[all_months].mean(axis=1)
std  = feat[all_months].std(axis=1)
feat["cv_mensuel"] = np.where(mean > 0, (std / (mean + 1e-9)), 0.0)

# Stabilité = 1 - CV normalisé
# Normalisation min-max sur l'ensemble des communes
cv_min, cv_max = feat["cv_mensuel"].min(), feat["cv_mensuel"].max()
feat["cv_norm"] = (feat["cv_mensuel"] - cv_min) / (cv_max - cv_min + 1e-9)
feat["stabilite"] = 1 - feat["cv_norm"]  # plus stable → score ↑

# Remettre en DataFrame plat
feat = feat.reset_index()
feat = feat[["code_commune","nom_commune","nb_ventes_2024","nb_mois_actifs","stabilite"]]

# Rangs percentile (0..1)
feat["rk_volume"] = feat["nb_ventes_2024"].rank(pct=True)
feat["rk_mois"]   = feat["nb_mois_actifs"].rank(pct=True)
feat["rk_stab"]   = feat["stabilite"].rank(pct=True)

# Pondérations 
w_vol, w_mois, w_stab = 0.60, 0.20, 0.20

feat["score_liquidite_2024"] = (w_vol*feat["rk_volume"] +
                                w_mois*feat["rk_mois"] +
                                w_stab*feat["rk_stab"]) * 100

# Classe qualitative
feat["classe_liquidite"] = pd.cut(
    feat["score_liquidite_2024"],
    bins=[-1,33,66,101],
    labels=["Illiquide","Intermédiaire","Liquide"],
    right=False
)


In [4]:
centroids = (df.groupby(["code_commune","nom_commune"], as_index=False)
                    .agg(latitude=("latitude","median"),
                         longitude=("longitude","median")))

liq_2024 = feat.merge(centroids, on=["code_commune","nom_commune"], how="left")
liq_2024.shape


df_map = liq_2024.copy()

@interact(
    score_min = IntSlider(min=0, max=100, step=5, value=60, description="Score min"),
    ventes_min = IntSlider(min=0, max=df_map["nb_ventes_2024"].max(), step=10, value=100, description="Ventes min")
)
def plot_liquidite(score_min, ventes_min):

    tmp = df_map[
        (df_map["score_liquidite_2024"] >= score_min) &
        (df_map["nb_ventes_2024"] >= ventes_min)
    ].copy()

    # Ajustement taille marqueur
    tmp["pt_size"] = (tmp["nb_ventes_2024"] / tmp["nb_ventes_2024"].max()) * 25 + 5

    fig = px.scatter_mapbox(
        tmp,
        lat="latitude",
        lon="longitude",
        color="score_liquidite_2024",
        size="pt_size",
        hover_name="nom_commune",
        hover_data=["nb_ventes_2024", "nb_mois_actifs", "stabilite"],
        zoom=9,
        height=650,
        center={"lat": 48.85, "lon": 2.35},
        color_continuous_scale="Turbo",   # ✅ meilleure lisibilité
    )

    fig.update_layout(
        mapbox_style="carto-positron",
        title="Indice de liquidité — Carte interactive (2024)",
        title_x=0.5,
        margin=dict(l=0, r=0, t=60, b=0),
        coloraxis_colorbar=dict(
            title="Liquidité",
            thickness=15,
            len=0.75,
        )
    )

    fig.update_traces(
        marker=dict(
            opacity=0.85,
        )
    )

    fig.show()


interactive(children=(IntSlider(value=60, description='Score min', step=5), IntSlider(value=100, description='…

In [ ]:
# Sélection top-10
top10 = (
    df_map
    .sort_values("score_liquidite_2024", ascending=False)
    .head(10)
)

# Bar chart horizontal
fig = px.bar(
    top10,
    x="score_liquidite_2024",
    y="nom_commune",
    orientation="h",
    color="score_liquidite_2024",
    color_continuous_scale="Viridis",
    labels={
        "nom_commune": "Commune",
        "score_liquidite_2024": "Score liquidité"
    },
    title="Top-10 — Communes les plus liquides (2024)",
)

# Afficher les valeurs
fig.update_traces(
    text=top10["score_liquidite_2024"].round(1).astype(str),
    textposition="outside"
)

# Réordonner y pour que le meilleur soit en haut
fig.update_yaxes(categoryorder="total ascending")

fig.show()


### Widget 6 : Analyser des accessibilité des transports en commun (par département, par commune)

### Widget 7 : Vérifier la sécurité et l’attractivité du quartier

### Widget 8: Trouver des biens accessibles avec son budget (par département, par commune)

### Widget 9 : Obtenir un classement des Top 10 opportunités actuelles (un tableau ou score automatique)